In [179]:
import requests
import pandas as pd
from datetime import datetime
import plotly.express as px

uni_get_data = requests.get(
    "https://api.coingecko.com/api/v3/coins/uniswap/market_chart?vs_currency=usd&days=max").json()

df_price = pd.DataFrame({
    'date': [datetime.utcfromtimestamp(value[0] / 1000) for value in uni_get_data['prices']], 
    'prices': [value[1] for value in uni_get_data['prices']]
})

px.line(df_price, x='date', y='prices', title='price')

In [212]:
df_tot_sup = pd.DataFrame({
    'date': [datetime.utcfromtimestamp(value[0] / 1000) for value in uni_get_data['total_volumes']], 
    'total_supply': [market_cap[1] / price[1] for price, market_cap in zip(uni_get_data['prices'], uni_get_data['total_volumes'])]
})

px.line(df_tot_sup, x='date', y='total_supply', title='total supply')

In [253]:
uni_get_data2 = requests.get('https://api.llama.fi/protocol/uniswap').json()

chainTvls = uni_get_data2['chainTvls']

df_chainTvls = None
for i, key in enumerate(chainTvls.keys()):
    tmp = pd.DataFrame(chainTvls[key]['tvl']).rename(columns={'totalLiquidityUSD': key})
    if i == 0:
        df_chainTvls = tmp
    else: 
        df_chainTvls = df_chainTvls.merge(tmp, on='date', suffixes=('', key), how='outer')
    
df_chainTvls['date'] = df_chainTvls.date.apply(lambda x: datetime.utcfromtimestamp(x))

px.line(df_chainTvls[df_chainTvls.date > '2022-01-01'], x='date', y=[x for x in df_chainTvls.columns if x != 'date'], 
      title='TVL')

In [262]:
uni_get_data3 = requests.get('https://api.coingecko.com/api/v3/exchanges/uniswap/volume_chart?days=500').json()

df_trading_volume = pd.DataFrame({
        'date': [datetime.utcfromtimestamp(value[0] / 1000) for value in uni_get_data3], 
        'value': [float(value[1]) for value in uni_get_data3],
    })

px.line(df_trading_volume, x='date', y='value', title='Trading Volumes')